In [1]:
pip install backtrader yfinance pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 6.5 MB/s eta 0:00:00


In [40]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import time
import os
import sys

# ==========================================
#  🔴  MASTER SWITCH  🔴
# ==========================================
TEST_ACCURACY = True    # Set TRUE to Backtest. Set FALSE to Scan Live.
# ==========================================

# --- CONFIGURATION ---
SYMBOLS = ["RELIANCE.NS", "TCS.NS", "INFY.NS", "HDFCBANK.NS", "ICICIBANK.NS", "LT.NS", "TITAN.NS", "BAJFINANCE.NS"]
CAPITAL = 1400        # Full Capital
TIMEFRAME = "1d"        # Daily Charts
STOP_LOSS_PCT = 0.05    # Hard 5% Stop Loss (Safety Net)

# --- NOTIFICATION ---
def send_notification(title, message):
    print(f"\n🔔 {title.upper()}: {message}")
    if os.name == 'nt':
        import winsound
        try: winsound.Beep(1000, 1000)
        except: pass
    else:
        sys.stdout.write('\a'); sys.stdout.flush()

# --- TAX CALCULATOR ---
def calculate_taxes(buy_price, sell_price, quantity):
    turnover = (buy_price + sell_price) * quantity
    brokerage = 0
    stt = 0.001 * turnover
    txn_charge = 0.0000325 * turnover
    gst = 0.18 * (brokerage + txn_charge)
    stamp = 0.00015 * (buy_price * quantity)
    return brokerage + stt + txn_charge + gst + stamp

# --- DATA ENGINE ---
def get_data(symbol, period="2y"):
    try:
        df = yf.download(symbol, period=period, interval=TIMEFRAME, progress=False, auto_adjust=True)
        if df.empty: return None
        if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.droplevel(1)
        if df.index.tz is None: df.index = df.index.tz_localize('UTC').tz_convert('Asia/Kolkata')
        else: df.index = df.index.tz_convert('Asia/Kolkata')
        return df
    except: return None

# --- INDICATORS ---
def calculate_indicators(df):
    df['SMA_200'] = df['Close'].rolling(window=200).mean()

    # RSI 2
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=2).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=2).mean()
    rs = gain / loss
    df['RSI_2'] = 100 - (100 / (1 + rs))
    return df

# ==========================================
#  MODE 1: BACKTESTER (Single Shot)
# ==========================================
def run_backtest():
    print(f"\n--- TESTING V21 (SINGLE SHOT) ---")
    print(f"Capital: ₹{CAPITAL} | Position: 100% Allocation")
    print(f"Strategy: Buy RSI(2) < 10 | Sell RSI(2) > 90")
    print(f"Stop Loss: {STOP_LOSS_PCT*100}%")

    data_map = {}
    for sym in SYMBOLS:
        print(f"Loading {sym}...", end="\r")
        df = get_data(sym)
        if df is not None:
            df = calculate_indicators(df)
            data_map[sym] = df

    print("\nRunning Simulation...")

    all_dates = set()
    for df in data_map.values(): all_dates.update(df.index)
    sorted_dates = sorted(list(all_dates))

    if len(sorted_dates) < 200: return
    start_date = sorted_dates[200]

    balance = CAPITAL
    active_position = None # {sym, qty, entry}
    trade_log = []

    for d in sorted_dates:
        if d < start_date: continue

        # A. CHECK EXIT (If holding a stock)
        if active_position:
            sym = active_position['sym']
            df = data_map[sym]
            if d not in df.index: continue

            row = df.loc[d]
            entry_price = active_position['entry']

            # EXIT LOGIC:
            # 1. Profit: RSI(2) > 90
            # 2. Safety: Price drops 5% below entry

            pnl_pct = (row['Close'] - entry_price) / entry_price

            target_hit = row['RSI_2'] > 90
            sl_hit = pnl_pct < -STOP_LOSS_PCT

            if target_hit or sl_hit:
                # Sell at Close (End of Day)
                exit_price = row['Close']
                qty = active_position['qty']

                gross = (exit_price - entry_price) * qty
                taxes = calculate_taxes(entry_price, exit_price, qty)
                net = gross - taxes

                balance += (entry_price * qty) + net

                reason = "Target Hit" if target_hit else "Stop Loss"

                trade_log.append({
                    'Date': d.date(), 'Symbol': sym, 'Type': 'SELL',
                    'Reason': reason, 'PnL': net, 'Return%': (net/(entry_price*qty))*100
                })
                active_position = None # Free up capital

        # B. CHECK ENTRY (If cash is free)
        if active_position is None:
            for sym in SYMBOLS:
                if active_position: break # Only 1 stock at a time

                df = data_map[sym]
                if d not in df.index: continue

                try:
                    loc = df.index.get_loc(d)
                    prev = df.iloc[loc-1]
                except: continue

                # ENTRY LOGIC: Uptrend + Dip
                if prev['Close'] > prev['SMA_200'] and prev['RSI_2'] < 10:

                    # 100% Allocation
                    entry_price = df.loc[d]['Open']
                    qty = int(balance / entry_price)

                    if qty > 0:
                        cost = qty * entry_price
                        balance -= cost
                        active_position = {'sym': sym, 'qty': qty, 'entry': entry_price}

                        trade_log.append({
                            'Date': d.date(), 'Symbol': sym, 'Type': 'BUY',
                            'Price': entry_price
                        })

    # REPORT
    df_res = pd.DataFrame(trade_log)

    final_equity = balance
    if active_position:
        last_price = data_map[active_position['sym']].iloc[-1]['Close']
        final_equity += active_position['qty'] * last_price

    total_pnl = final_equity - CAPITAL

    print("\n" + "="*40)
    print(f"INITIAL CAPITAL: ₹{CAPITAL}")
    print(f"FINAL EQUITY:    ₹{final_equity:.2f}")
    print(f"NET PNL:         ₹{total_pnl:.2f}")
    print(f"ROI:             {(total_pnl/CAPITAL)*100:.2f}%")

    sells = df_res[df_res['Type'] == 'SELL']
    if not sells.empty:
        wins = len(sells[sells['PnL'] > 0])
        print(f"WIN RATE:        {(wins/len(sells))*100:.2f}%")
        print(f"TOTAL TRADES:    {len(sells)}")
        print("\nLast 5 Completed Trades:")
        print(sells[['Date', 'Symbol', 'Reason', 'Return%']].tail())
    else:
        print("No completed trades.")
    print("="*40)

# ==========================================
#  MODE 2: LIVE SCANNER
# ==========================================
def run_live_scanner():
    print(f"\n--- V21 LIVE SCANNER (SINGLE SHOT) ---")
    print("Scan daily at 3:15 PM. Buy Delivery (CNC).")

    for sym in SYMBOLS:
        df = get_data(sym)
        if df is None: continue
        df = calculate_indicators(df)

        curr = df.iloc[-1]
        trend = "UP" if curr['Close'] > curr['SMA_200'] else "DOWN"
        rsi = curr['RSI_2']

        print(f"{sym:<12} | RSI(2): {rsi:.1f} | Trend: {trend}")

        if trend == "UP" and rsi < 10:
            send_notification("BUY SIGNAL", f"Buy {sym} (RSI {rsi:.1f})")
            print(f">>> 🚨 BUY SIGNAL: {sym}")

if __name__ == "__main__":
    if TEST_ACCURACY:
        run_backtest()
    else:
        run_live_scanner()


--- TESTING V21 (SINGLE SHOT) ---
Capital: ₹1400 | Position: 100% Allocation
Strategy: Buy RSI(2) < 10 | Sell RSI(2) > 90
Stop Loss: 5.0%

Running Simulation...

INITIAL CAPITAL: ₹1400
FINAL EQUITY:    ₹1571.40
NET PNL:         ₹171.40
ROI:             12.24%
WIN RATE:        70.69%
TOTAL TRADES:    58

Last 5 Completed Trades:
           Date         Symbol      Reason   Return%
107  2025-11-11    RELIANCE.NS  Target Hit  0.410097
109  2025-11-17  BAJFINANCE.NS  Target Hit  1.188214
111  2025-11-27  BAJFINANCE.NS  Target Hit  1.627628
113  2025-12-08    HDFCBANK.NS  Target Hit  1.304081
115  2025-12-12    HDFCBANK.NS  Target Hit  0.177916
